In [ ]:
import os
import requests
import json
from datetime import datetime
from dotenv import dotenv_values

# 환경 변수 파일에서 국회 API 키 읽기
def load_api_key():
    config = dotenv_values(
        dotenv_path = "../.env"
    )
    
    api_key = config.get('open_national_assembly_information')
    
    if api_key:
        print("국회 API 키가 성공적으로 로드되었습니다.")
        return api_key
    else:
        print("국회 API 키를 찾을 수 없습니다. .env 파일을 확인해주세요.")
        return None

# 국회 Open API를 사용하여 예산안 데이터 수집
def fetch_budget_bills(api_key, page_size=10, page_index=1):
    """
    국회 의안정보 API를 통해 예산안 관련 데이터를 수집합니다.
    """
    
    # API 기본 URL (의안정보 서비스)
    base_url = "https://open.assembly.go.kr/portal/openapi/nwvrqwxyaytdsfvhu"
    
    # 요청 파라미터
    params = {
        "KEY": api_key,
        "Type": "json",           # 응답 형식 (json/xml)
        "pIndex": page_index,     # 페이지 번호
        "pSize": page_size,       # 페이지당 건수
        "BILL_NAME": "예산",       # 의안명에 '예산' 포함
    }
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        
        # JSON 응답 파싱
        data = response.json()
        
        # 응답 구조 확인
        if "nwvrqwxyaytdsfvhu" in data and len(data["nwvrqwxyaytdsfvhu"]) > 1:
            # 헤더 정보 (첫 번째 항목)
            header = data["nwvrqwxyaytdsfvhu"][0]["head"]
            print(f"총 건수: {header[0]['list_total_count']}")
            
            # 실제 데이터 (두 번째 항목부터)
            bills = data["nwvrqwxyaytdsfvhu"][1]["row"]
            return bills
        else:
            print("응답 데이터가 예상과 다릅니다.")
            print(json.dumps(data, indent=2, ensure_ascii=False))
            return []
            
    except requests.exceptions.RequestException as e:
        print(f"API 요청 중 오류 발생: {e}")
        return []
    except json.JSONDecodeError as e:
        print(f"JSON 파싱 오류: {e}")
        return []

# 예산안 데이터 상세 출력
def display_budget_bills(bills):
    """
    예산안 데이터를 보기 좋게 출력합니다.
    """
    for i, bill in enumerate(bills, 1):
        print(f"\n=== 예산안 {i} ===")
        print(f"의안명: {bill.get('BILL_NAME', 'N/A')}")
        print(f"의안번호: {bill.get('BILL_NO', 'N/A')}")
        print(f"제안일자: {bill.get('PROPOSE_DT', 'N/A')}")
        print(f"제안자: {bill.get('PROPOSER', 'N/A')}")
        print(f"위원회: {bill.get('COMMITTEE', 'N/A')}")
        print(f"처리상태: {bill.get('PROC_RESULT', 'N/A')}")
        print(f"요약: {bill.get('SUMMARY', 'N/A')[:100]}...")
        print("-" * 80)

# 실행 예시
if __name__ == "__main__":
    # API 키 로드
    api_key = load_api_key()
    
    if api_key:
        # 예산안 데이터 수집
        print("예산안 데이터를 수집하고 있습니다...")
        budget_bills = fetch_budget_bills(api_key, page_size=5)
        
        if budget_bills:
            display_budget_bills(budget_bills)
        else:
            print("예산안 데이터를 찾을 수 없습니다.")
    else:
        print("API 키가 없어서 실행할 수 없습니다.")


In [ ]:
config = dotenv_values(
    dotenv_path = "../.env"
)

api_key = config.get('open_national_assembly_information')
    
base_url = "https://open.assembly.go.kr/portal/openapi/nwvrqwxyaytdsfvhu"

# 요청 파라미터 
params = {
    "KEY": api_key,
    "Type": "json",           # 응답 형식 (json/xml)
    "pIndex": 1,     # 페이지 번호
    "pSize": 10,       # 페이지당 건수
    "BILL_NAME": "예산",       # 의안명에 '예산' 포함
}

response = requests.get(base_url, params=params)

In [ ]:
response.json()